In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('dataset.csv')

df

/var/folders/vd/8wp67d_96s523426gf09w0k80000gn/T/ipykernel_8160/4083470388.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset.csv')


,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558
0,0,125,125,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
1,1,57,468,8.2105,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
2,2,33,230,6.9696,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
3,3,60,468,7.8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
4,4,60,468,7.8,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ad.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3274,3274,170,94,0.5529,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
3275,3275,101,140,1.3861,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
3276,3276,23,120,5.2173,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.
3277,3277,?,?,?,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,nonad.


In [3]:
print('Liczba wierszy: ', df.shape[0])
print('Liczba kolumn: ', df.shape[1])

Liczba wierszy:  3279
Liczba kolumn:  1560


In [7]:
df.groupby('1558').size()

1558
ad.        459
nonad.    2820
dtype: int64